In [ ]:
pip install wget

In [ ]:
import astropy
from astropy.table import Table,vstack, Column
from subprocess import call
import subprocess
from astropy.io import fits
import numpy as np
import wget
import matplotlib.pyplot as plt
from gpz_pype import utilities
from gpz_pype.utilities import Params, set_gpz_path, basic_lupt_soft, flux_to_lupt
import astropy.units as u
from numpy.ma import masked
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky

from os import listdir
from os.path import isfile, join

In [ ]:
#jwst fields
fields= ['gds','ngdeep','primer-cosmos-east','primer-cosmos-west','primer-uds-north','abell2744clu','abells1063','elgordo','j112716p4228','macs0647','rxj2129','smacs0723','sunrise']

# Download Field Files


In [ ]:
url=f'https://s3.amazonaws.com/grizli-v2/JwstMosaics/v7/abell2744clu-grizli-v7.2-fix.eazypy.zout.fits'
filename=wget.download(url)


In [ ]:
hdul= fits.open('gdn-grizli-v7.0_phot.fits', memmap=True,formats='fits')
t = Table.read(hdul)

In [ ]:
jwst_merge= Table.read('MPhys/cut_flux_photometry_only.fits',format='fits')


# Initial Catalogue Making - Run Once


In [ ]:
###INITIAL CATALOGUE MAKING
hdu_list = fits.open('ceers-full-grizli-v7.0_phot.fits', memmap=True,formats='fits')
ceers = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('gdn-grizli-v7.0_phot.fits', memmap=True,formats='fits')
gdn= Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('gds-grizli-v7.0_phot.fits', memmap=True,formats='fits')
gds = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('ngdeep-grizli-v7.0_phot.fits', memmap=True,formats='fits')
ngdeep = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('primer-cosmos-east-grizli-v7.0_phot.fits', memmap=True,formats='fits')
cosmos_east = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('primer-cosmos-west-grizli-v7.0_phot.fits', memmap=True,formats='fits')
cosmos_west = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('primer-uds-south-grizli-v7.0_phot.fits', memmap=True,formats='fits')
udss = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('primer-uds-north-grizli-v7.0_phot.fits', memmap=True,formats='fits')
udsn = Table(hdu_list[1].data)
hdu_list.close()

In [ ]:
#ADDING FIELDS TO CATALOGUE - RUN BOTH CELLS IF STARTING FROM SCRATCH
hdu_list = fits.open('abell2744clu-grizli-v7.0_phot.fits', memmap=True,formats='fits')
abell2744 = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('abells1063-grizli-v7.0_phot.fits', memmap=True,formats='fits')
abells1063= Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('elgordo-grizli-v7.0_phot.fits', memmap=True,formats='fits')
elgordo = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('j112716p4228-grizli-v7.0_phot.fits', memmap=True,formats='fits')
j_field = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('macs0647-grizli-v7.0_phot.fits', memmap=True,formats='fits')
macs = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('rxj2129-grizli-v7.0_phot.fits', memmap=True,formats='fits')
rxj = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('smacs0723-grizli-v7.0_phot.fits', memmap=True,formats='fits')
smacs = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('sunrise-grizli-v7.0_phot.fits', memmap=True,formats='fits')
sunrise = Table(hdu_list[1].data)
hdu_list.close()

In [ ]:
abell2744.add_column('abells2744clu', name='field', index=0)
abells1063.add_column('abells1063', name='field', index=0)
elgordo.add_column('elgordo', name='field', index=0)
j_field.add_column('j112716p4228', name='field', index=0)
macs.add_column('macs0647', name='field', index=0)
rxj.add_column('rxj2129', name='field', index=0)
smacs.add_column('smacs0723', name='field', index=0)
sunrise.add_column('sunrise', name='field', index=0)

In [ ]:
ceers.add_column('ceers', name='field', index=0)
gdn.add_column('gdn', name='field', index=0)
gds.add_column('gds', name='field', index=0)
ngdeep.add_column('ngdeep', name='field', index=0)
cosmos_east.add_column('cosmos_east', name='field', index=0)
cosmos_west.add_column('cosmos_west', name='field', index=0)
udsn.add_column('udsn', name='field', index=0)
udss.add_column('udss', name='field', index=0)

In [ ]:
main = vstack([ceers,gdn,gds,ngdeep,cosmos_east,cosmos_west,udsn,udss])

In [ ]:
side = vstack([abell2744,abells1063,elgordo,j_field,macs,rxj,smacs,sunrise])

In [ ]:
main.write('MPhys/original_jwst_data/main_merged.fits',format='fits',overwrite=True)

In [ ]:
side.write('MPhys/original_jwst_data/other_merged.fits',format='fits',overwrite=True)

In [ ]:
all_merge = vstack([m4,side])

In [ ]:
jwst_merge.write('MPhys/original_jwst_data/all_merged_ab.fits',format='fits',overwrite=True)

In [ ]:
jwst_merge = Table.read('MPhys/original_jwst_data/all_merged_ab.fits',format ='fits')

## CANDELS

In [ ]:
cosmos = Table.read('MPhys/CANDELS/CANDELS.COSMOS.F160W.Processed.photz.fits',format='fits')
egs = Table.read('MPhys/CANDELS/CANDELS.EGS.F160W.Processed.photz.fits',format='fits')
uds_candels = Table.read('MPhys/CANDELS/CANDELS.UDS.F160W.Processed.photz.fits',format='fits')
gds_candels = Table.read('MPhys/CANDELS/CANDELS.GOODSS.F160W.Processed.photz.fits',format='fits')
gdn_candels = Table.read('MPhys/CANDELS/CANDELS.GOODSN.F160W.Processed.photz.fits',format='fits')

In [ ]:
candels = vstack([cosmos,egs,uds_candels,gds_candels,gdn_candels])

In [ ]:
candels.write('MPhys/CANDELS/candels_combined.fits',format='fits')

# Convert to Luptiudes for Each Field


In [ ]:
field=gds


In [ ]:
filters0 = [col.split('_')[0] for col in field.colnames if col.endswith('flux_aper_0')]
filters0=filters0[1:]
filters1 = [col.split('_')[0] for col in field.colnames if col.endswith('flux_aper_1')]
filters1=filters1[1:]

filters2 = [col.split('_')[0] for col in field.colnames if col.endswith('flux_aper_2')]
filters2=filters2[1:]

filters3 = [col.split('_')[0] for col in field.colnames if col.endswith('flux_aper_3')]
filters3=filters3[1:]



In [ ]:

b_arr0 = [basic_lupt_soft(field[f'{filt}_flux_aper_0'], field[f'{filt}_fluxerr_aper_0']) for filt in filters0] 
b_arr1 = [basic_lupt_soft(field[f'{filt}_flux_aper_1'], field[f'{filt}_fluxerr_aper_1']) for filt in filters1] 
b_arr2 = [basic_lupt_soft(field[f'{filt}_flux_aper_2'], field[f'{filt}_fluxerr_aper_2']) for filt in filters2] 
b_arr3 = [basic_lupt_soft(field[f'{filt}_flux_aper_3'], field[f'{filt}_fluxerr_aper_3']) for filt in filters3] 

In [ ]:
i=2
lupt_cols=field['field','id','ra','dec']

for filt,b in zip(filters0,b_arr0):
    lupt,lupterr = flux_to_lupt(field[f'{filt}_flux_aper_0'],field[f'{filt}_fluxerr_aper_0'],b)
    lupt_cols[f'lupt_{filt}_aper0']=lupt
    lupt_cols[f'lupterr_{filt}_aper0']=lupterr
for filt,b in zip(filters1,b_arr1):
    lupt,lupterr = flux_to_lupt(field[f'{filt}_flux_aper_1'],field[f'{filt}_fluxerr_aper_1'],b)
    lupt_cols[f'lupt_{filt}_aper1']=lupt
    lupt_cols[f'lupterr_{filt}_aper1']=lupterr
for filt,b in zip(filters2,b_arr2):
    lupt,lupterr = flux_to_lupt(field[f'{filt}_flux_aper_2'],field[f'{filt}_fluxerr_aper_2'],b)
    lupt_cols[f'lupt_{filt}_aper2']=lupt
    lupt_cols[f'lupterr_{filt}_aper2']=lupterr
for filt,b in zip(filters3,b_arr3):
    lupt,lupterr = flux_to_lupt(field[f'{filt}_flux_aper_3'],field[f'{filt}_fluxerr_aper_3'],b)
    lupt_cols[f'lupt_{filt}_aper3']=lupt
    lupt_cols[f'lupterr_{filt}_aper3']=lupterr

lupt_cols.write(f'MPhys/catalogues/{fields[i]}_lupts.fits',format='fits', overwrite=True)

In [ ]:
t = Table.read('MPhys/catalogues/gds_lupts.fits',format ='fits')
t

In [ ]:
lup_ceers=Table.read('MPhys/catalogues/ceers-full_lupts.fits',format='fits')
lup_gds=Table.read('MPhys/catalogues/gds_lupts.fits',format='fits')
lup_gdn=Table.read('MPhys/catalogues/gdn_lupts.fits',format='fits')
lup_ngdeep=Table.read('MPhys/catalogues/ngdeep_lupts.fits',format='fits')
lup_udss=Table.read('MPhys/catalogues/primer-uds-south_lupts.fits',format='fits')
lup_udsn=Table.read('MPhys/catalogues/primer-uds-north_lupts.fits',format='fits')
lup_cosmos_east=Table.read('MPhys/catalogues/primer-cosmos-east_lupts.fits',format='fits')
lup_cosmos_west=Table.read('MPhys/catalogues/primer-cosmos-west_lupts.fits',format='fits')

In [ ]:
lup_abell2744clu = Table.read('MPhys/catalogues/abell2744clu_lupts.fits',format='fits')
lup_abells1063=Table.read('MPhys/catalogues/abells1063_lupts.fits',format='fits')
lup_elgordo=Table.read('MPhys/catalogues/elgordo_lupts.fits',format='fits')
lup_j112716p4228=Table.read('MPhys/catalogues/j112716p4228_lupts.fits',format='fits')
lup_macs0647=Table.read('MPhys/catalogues/macs0647_lupts.fits',format='fits')
lup_rxj2129=Table.read('MPhys/catalogues/rxj2129_lupts.fits',format='fits')
lup_smacs0723=Table.read('MPhys/catalogues/smacs0723_lupts.fits',format='fits')
lup_sunrise=Table.read('MPhys/catalogues/sunrise_lupts.fits',format='fits')

In [ ]:
lup_con =vstack([lup_ceers,lup_gds,lup_gdn,lup_ngdeep,lup_udss,lup_udsn,lup_cosmos_east,lup_cosmos_west,lup_abell2744clu])


In [ ]:
lup_con_small = vstack([lup_abell2744clu,lup_abells1063,lup_elgordo,lup_j112716p4228,lup_macs0647,lup_rxj2129,lup_smacs0723,lup_sunrise])

In [ ]:
lup_total = vstack([lup_con,lup_con_small])

In [ ]:
lup_total.write('luptitudes_all_jwst.fits', format='fits', overwrite=True)

In [ ]:
t = Table.read('luptitudes_all_jwst.fits', format='fits')
t.add_column(np.nan, name='z_spec',index =4)

In [ ]:
print(len(t))
new_id = np.arange(0,len(t),1)
print(new_id)
t['id']= new_id

In [ ]:
t


# Skymatching Catalogues
Matching the astrophysics coordinates (RA, Dec) of each source in different fits files to then assign the spectroscopic redshift to the photometry

### JWST match to CANDELS

In [ ]:
candels=Table.read('MPhys/candels_combined.fits',format='fits')
#jwst_merge=Table.read('MPhys/original_jwst_data/all_merged_ab.fits',format = 'fits')



In [ ]:
#matching jwst and candels
ra1=jwst_merge['ra']

dec1=jwst_merge['dec']
ra2=candels['RA']
dec2=candels['DEC']
print(len(ra2))

In [ ]:
jwst_cat = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree)#dawn
candels_cat = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree)#candels


In [ ]:
max_sep = 0.1 * u.arcsec
idx, d2d, d3d = jwst_cat.match_to_catalog_sky(candels_cat)#catalog = candels information
idx_jwst, d2d_jwst, d3d_jwst = candels_cat.match_to_catalog_sky(jwst_cat)
#idx, d2d, d3d = jwst_cat.match_to_catalog_sky(candels_cat)
print(len(idx_jwst))

In [ ]:
jwst_merge.add_column(np.nan, name= 'z_spec',index=4)

In [ ]:
c_index=[]
j_index=[]
for i in range(len(candels)):
    if abs(ra2[i]-ra1[idx_jwst[i]])<0.1:
        c_index.append(i)
        j_index.append(idx_jwst[i])
        jwst_merge['z_spec'][idx_jwst[i]]=candels['z_spec'][i]


### JWST match to DAWN

In [ ]:
dawn = Table.read('MPhys/dja_nirspec_msa_extractions.csv',format='csv')

In [ ]:
grade_good = np.zeros(len(dawn))
for i in range(len(dawn)):
    if dawn['grade'][i]==3:
        grade_good[i]=1
        
dawn = dawn[np.array(grade_good,dtype=bool)]

In [ ]:

ra3 = jwst_merge['ra']
dec3=jwst_merge['dec']
ra4=dawn['ra']
dec4=dawn['dec']
j_merge_cat = SkyCoord(ra=ra4*u.degree, dec=dec4*u.degree)
dawn_cat = SkyCoord(ra=ra4*u.degree, dec=dec4*u.degree)

# EXCELS

In [ ]:
idx_all, d2d_all, d3d_all = dawn_cat.match_to_catalog_sky(j_merge_cat)

In [ ]:

for i in range(len(dawn)):
    if abs(ra4[i]-ra3[idx_all[i]])<0.1:
        jwst_merge['z_spec'][idx_all[i]]=dawn['z'][i]


In [ ]:
excels = Table.read(filename,format='ascii') #using filename instead of file as data is not public yet
flag = np.zeros(len(excels))
for i in range(len(excels)):
    if excels['zflag_excels_acc'][i]==3 or excels['zflag_excels_acc'][i]==4 or excels['zflag_excels_acc'][i]==9:
        flag[i]=1
excels = excels[np.array(flag,dtype=bool)]

In [ ]:
jwst_merge['z_spec'][jwst_merge['z_spec']<=0]=np.ma.masked

In [ ]:
jwst_merge.write('MPhys/original_jwst_data/final_skymatched_catalogue.fits',format = 'fits',overwrite=True)

# Filter Choices for Training Catalogue
Need to choose common filters to construct training catalogue for GPz++


In [ ]:

train = Table.read('MPhys/gpz_catalogues/final_skymatched_abmag_catalogue.fits',format = 'fits')


In [ ]:
#Full catalogue filter names 
filters_1 = [col.split('_')[1] for col in train.colnames if col.endswith('_aper1')]


In [ ]:
#### FILTER CHOICES
bc= ['f115w-clear','f200w-clear','f356w-clear','f444w-clear']

#swap f115w for f814w,f606w,f435w,f125w,f140w,f160w,f150w
c1 =['f814w','f200w-clear','f356w-clear','f444w-clear']
c2 =['f606w','f200w-clear','f356w-clear','f444w-clear']
c3 =['f435w','f200w-clear','f356w-clear','f444w-clear']
c4 =['f125w','f200w-clear','f356w-clear','f444w-clear']
c5 =['f140w','f200w-clear','f356w-clear','f444w-clear']
c6 =['f160w','f200w-clear','f356w-clear','f444w-clear']
c7 =['f150w-clear','f200w-clear','f356w-clear','f444w-clear']

#swap f444w for f410m
c9= ['f115w-clear','f200w-clear','f356w-clear','f410m-clear']
#other choices 
c10=['f115w-clear','f606w','f150w-clear','f444w-clear']
bc_extra = ['f090w-clear','f115w-clear','f200w-clear','f356w-clear','f444w-clear']

#order of swaps
swap= ['bc','f814w','f606w','f435w','f125w','f140w','f160w','f150w-clear','f410m-clear','ic','f090w_inc']


In [ ]:
######change this to make new catalogue of filters and can use aperture sizes 0-4
aper=1
#aper_size=1

choice=bc
num = 0

print(swap[num])
lup_train = train['field','id','ra','dec','z_spec']
lup_train[f'lupt_{choice[0]}']=train[f'lupt_{choice[0]}_aper{aper}']
lup_train[f'lupterr_{choice[0]}']=train[f'lupterr_{choice[0]}_aper{aper}']
lup_train[f'lupt_{choice[1]}']=train[f'lupt_{choice[1]}_aper{aper}']
lup_train[f'lupterr_{choice[1]}']=train[f'lupterr_{choice[1]}_aper{aper}']
lup_train[f'lupt_{choice[2]}']=train[f'lupt_{choice[2]}_aper{aper}']
lup_train[f'lupterr_{choice[2]}']=train[f'lupterr_{choice[2]}_aper{aper}']
lup_train[f'lupt_{choice[3]}']=train[f'lupt_{choice[3]}_aper{aper}']
lup_train[f'lupterr_{choice[3]}']=train[f'lupterr_{choice[3]}_aper{aper}']

#include if using more than 4 filters

#lup_train[f'lupt_{choice[4]}']=train[f'lupt_{choice[4]}_aper{aper}']
#lup_train[f'lupterr_{choice[4]}']=train[f'lupterr_{choice[4]}_aper{aper}']

#Adds size column 
#lup_train[f'lupt_{choice[3]}_size']=train[f'lupt_{choice[3]}_aper{aper_size}']/train[f'lupt_{choice[3]}_aper{aper}']
#lup_train[f'lupterr_{choice[3]}_size']=train[f'lupterr_{choice[3]}_aper{aper_size}']/train[f'lupterr_{choice[3]}_aper{aper}']

In [ ]:
#removes row if cell is masked
#repeated for each filter to ensure final catalogue has full photometry

f1=[]
for i in range(len(lup_train)):
    f1.append(np.ma.is_masked(lup_train[f'lupt_{choice[0]}'][i]))
f1_inv=np.array(np.invert(f1))
f1_cat =lup_train[f1_inv*(np.ma.is_masked(lup_train[f'lupt_{choice[0]}']) == True)] 

In [ ]:

f2=[]
for i in range(len(f1_cat)):
    f2.append(np.ma.is_masked(f1_cat[f'lupt_{choice[1]}'][i]))
f2_inv=np.array(np.invert(f2))
f2_cat =f1_cat[f2_inv*(np.ma.is_masked(f1_cat[f'lupt_{choice[1]}']) ==True)] 

In [ ]:

f3=[]
for i in range(len(f2_cat)):
    f3.append(np.ma.is_masked(f2_cat[f'lupt_{choice[2]}'][i]))
f3_inv=np.array(np.invert(f3))
f3_cat =f2_cat[f3_inv*(np.ma.is_masked(f2_cat[f'lupt_{choice[2]}']) == True)] 

In [ ]:

f4=[]
for i in range(len(f3_cat)):
    f4.append(np.ma.is_masked(f3_cat[f'lupt_{choice[3]}'][i]))
f4_inv=np.array(np.invert(f4))
f4_cat =f3_cat[f4_inv*(np.ma.is_masked(f3_cat[f'lupt_{choice[3]}']) == True)] #false idk why but works 


In [ ]:
#USE IF USING 5TH FILTER
f5=[]
for i in range(len(f4_cat)):
    f5.append(np.ma.is_masked(f4_cat[f'lupt_{choice[4]}'][i]))
f5_inv=np.array(np.invert(f5))
f5_cat =f4_cat[f5_inv*(np.ma.is_masked(f4_cat[f'lupterr_{choice[4]}']) ==True)] 

In [ ]:
f4_cat.write(f'MPhys/gmm_files/{swap[num]}_luptitude_full_catalogue.fits',format='fits',overwrite=True)
#f5_cat.write(f'MPhys/base_filter_permutations/base_filter_permutations/size/informed_choice_size.cat',format = 'ascii.commented_header',overwrite=True)

In [ ]:
f5_cat['z_spec'][f5_cat['z_spec']==0]=np.nan

In [ ]:
#MAKE THE TRAINING CATALOGUE
#USE IF USING 5TH FILTER
training=[]
for i in range(len(f5_cat)):
    training.append(np.isnan(f5_cat['z_spec'][i]))
training_inv=np.array(np.invert(training))
training_cat =f5_cat[training_inv*(np.isnan(f5_cat['z_spec']) == False)] 

In [ ]:
training_cat.write(f'MPhys/gpz_catalogues/{swap[num]}_full_training_cat.cat',format='ascii.commented_header',overwrite=True)

# Adding Size Column 

In [ ]:
cat = Table.read('MPhys/gmm_cats/input_catalogues/f160w_extra_apertures_full_catalogue.fits',format='fits')
catalogue = Table.read('MPhys/gmm_cats/input_catalogues/f160w_luptitude_full_catalogue_new.fits',format='fits')
size = cat['lupt_f444w-clear']/cat['lupt_f444w-clear_aper2']
size_err =  cat['lupterr_f444w-clear']/cat['lupterr_f444w-clear_aper2']
catalogue.add_column(size,name=('lupt_f444w-clear_size'))
catalogue.add_column(size_err,name=('lupterr_f444w-clear_size'))

In [ ]:
cat = Table.read(f'MPhys/gmm_cats/input_catalogues/f410m-clear_extra_apertures_full_catalogue.fits',format='fits')

# Pred/Train Split
#### Set at 70:15:15 split

GPz++ has inbuilt pred/valid split (set at 50% currently) so only need to split this section into tra9ning and prediction



In [ ]:
filt= ['bc','f160w','f435w','f814w','f606w','f435w','f125w','f140w','f160w','f150w-clear','f410m-clear','ic','f090w_inc']


In [ ]:
for f in filt:
    print(f)
    #t_full =Table.read(f'MPhys/gpz_catalogues/gpz_catalogues/{swap[num]}_full_training_cat_new.cat',format='ascii.commented_header')
    #cat = Table.read(f'MPhys/gmm_cats/input_catalogues/{f}_extra_apertures_full_catalogue.fits',format='fits')
    catalogue = Table.read(f'MPhys/gmm_cats/input_catalogues/{f}_luptitude_full_catalogue_new.fits',format='fits')
    #size = cat['lupt_f410m-clear']/cat['lupt_f410m-clear_aper2']
    #size_err =  cat['lupterr_f410m-clear']/cat['lupterr_f410m-clear_aper2']
    #catalogue.add_column(size,name=('lupt_f410m-clear_size'))
    #catalogue.add_column(size_err,name=('lupterr_f410m-clear_size'))
    t_full = catalogue
    #t_full.sort('z_spec')
    z= t_full['z_spec']
    bins = np.linspace(np.min(z),np.max(z),5)
    h2 = np.zeros(len(z))
    h3 = np.zeros(len(z))
    h4 = np.zeros(len(z))
    low = np.zeros(len(z))

    for i in range(len(z)):
        if bins[0]<=z[i]<=bins[1]:
            low[i]=1
        elif bins[1]<=z[i]<=bins[2]:
            h2[i]=1
        elif bins[2]<=z[i]<=bins[3]:
            h3[i]=1
        elif bins[3]<=z[i]<=bins[4]:
            h4[i]=1
    h2_bool = np.array(h2, dtype =bool)
    h3_bool = np.array(h3, dtype =bool)
    h4_bool = np.array(h4, dtype =bool)
    low_bool = np.array(low, dtype =bool)

    low_cat = t_full[low_bool]
    h2_cat = t_full[h2_bool]
    h3_cat = t_full[h3_bool]
    h4_cat = t_full[h4_bool]
    h2_lim = int(len(h2_cat)*0.7)
    h3_lim = int(len(h3_cat)*0.7)
    h4_lim = int(len(h4_cat)*0.7)
    low_lim = int(len(low_cat)*0.7)

    t_low = low_cat[0:low_lim]
    p_low = low_cat[low_lim:len(low_cat)]

    t_h2 = h2_cat[0:h2_lim]
    p_h2 = h2_cat[h2_lim:len(h2_cat)]
    t_h3 = h3_cat[0:h3_lim]
    p_h3 = h3_cat[h3_lim:len(h3_cat)]
    t_h4 = h4_cat[0:h4_lim]
    p_h4 = h4_cat[h4_lim:len(h4_cat)]


    t_high = vstack([t_h2,t_h3,t_h4])
    p_high = vstack([p_h2,p_h3,p_h4])

    t_all = vstack([t_low,t_high])
    p_all = vstack([p_low,p_high])

    t_low.write(f'MPhys/gpz_catalogues/gpz_catalogues/reg_z_split/lowz/TRAIN/new/train_lowz_{f}.cat',format = 'ascii.commented_header',overwrite=True)
    p_low.write(f'MPhys/gpz_catalogues/gpz_catalogues/reg_z_split/lowz/PRED/new/pred_lowz_{f}.cat',format = 'ascii.commented_header',overwrite=True)
    t_high.write(f'MPhys/gpz_catalogues/gpz_catalogues/reg_z_split/highz/TRAIN/new/train_highz_{f}.cat',format = 'ascii.commented_header',overwrite=True)
    p_high.write(f'MPhys/gpz_catalogues/gpz_catalogues/reg_z_split/highz/PRED/new/pred_highz_{f}.cat',format = 'ascii.commented_header',overwrite=True)

    t_all.write(f'MPhys/gpz_catalogues/gpz_catalogues/reg/TRAIN/new/train_{f}.cat',format = 'ascii.commented_header',overwrite=True)
    p_all.write(f'MPhys/gpz_catalogues/gpz_catalogues/reg/PRED/new/pred_{f}.cat',format = 'ascii.commented_header',overwrite=True)

